In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
#read data
train_data = pd.read_csv('points.dat.txt',sep=' ', header=None)
train_data

,0,1
0,-6.367092,-0.352406
1,-0.089791,1.516617
2,0.228603,-3.400796
3,-5.373754,-0.346060
4,-0.252466,2.264652
...,...,...
995,-3.250677,-0.814128
996,-1.307515,1.123990
997,-2.554552,-4.718175
998,-0.201516,2.932802


In [3]:
def jointGaussianPdf(dataset, features_num, k, data_idx, mu, sigma):
    
    point = dataset[data_idx,:]- mu[k,:]
    a = np.power(2*np.pi,-features_num/2)*np.sqrt(np.linalg.det(sigma[:,:,k]))
    b = np.exp((-1/2)*np.matmul(point,np.matmul(np.linalg.inv(sigma[:,:,k]),np.transpose(point))))
                
    return a*b

In [4]:
def expectation(dataset,sample_num,k,r,pi,mu,sigma):
    #calculate joint gaussian pdf
    D = dataset.shape[1] #features
    Nk = np.zeros((k,1))
    for s in range(0,sample_num):
        for c in range(0,k):
            r[s,c] = np.multiply(pi[c,:],jointGaussianPdf(dataset,D,c,s,mu,sigma))
        
    for c in range(0,k):
        r[:,c] = np.divide(r[:,c],np.sum(r,axis=1))
        Nk[c,:] = np.sum(r[:,c],axis=0)
    return r, Nk

In [5]:
def maximizeMean(dataset,sample_num,k,r,Nk,mu):
    
    weightedSum = np.zeros((sample_num,dataset.shape[1]))
    for c in range(0,k):
        for s in range(0,sample_num):
            weightedSum[s,:] = r[s,c]*dataset[s,:]
        mu[c,:] = np.sum(weightedSum,axis=0)/Nk[c,:]
    
    return mu

In [6]:
def maximizeCovariance(dataset,sample_num,k,r,Nk,mu,sigma):
    
    weightedCov = np.zeros((dataset.shape[1],dataset.shape[1],k))
    for c in range(0,k):
        points = dataset - mu[c,:]
        weightedCov[:,:,c] = np.matmul(np.multiply(r[:,c],np.transpose(points)),points)
        sigma[:,:,c] = weightedCov[:,:,c]/Nk[c,:]
    return sigma

In [7]:
def maximizeMixture(k,sample_num,r,Nk,pi):
    for c in range(0,k):
        pi[c,:] = Nk[c,:]/sample_num
    return pi

In [8]:
def EM(dataset,sample_num,k,r,pi,mu,sigma,iter_num):
    D = dataset.shape[1] #features
    for i in range(0,iter_num):
        
        #1 evaluate the log-likelihood for sigma pi mu
        log_likelihood = np.zeros((sample_num,1))
        for d in range(0,sample_num):
            likelihood = 0
            for c in range(0,k):
                #evaluate conditional prob.(gaussian)
                likelihood += np.multiply(pi[c,:],jointGaussianPdf(dataset,D,c,d,mu,sigma))
            log_likelihood[d,:] = np.log(likelihood)
        #print log-likelihood for iteration i
        print('log-likelihood for iteration ',i,' is: ',np.sum(log_likelihood,axis=0))

        #2 E step: evaluate expectation given current pi, mu, sigma
        r, Nk = expectation(dataset,sample_num,k,r,pi,mu,sigma)
        #3 M step: evaluate new pi, mu, sigma
        mu = maximizeMean(dataset,sample_num,k,r,Nk,mu)
        sigma = maximizeCovariance(dataset,sample_num,k,r,Nk,mu,sigma)
        pi = maximizeMixture(k,sample_num,r,Nk,pi)
        
    return pi, mu, sigma

In [9]:
    #define variables
    k = 3 #clusters
    f = train_data.iloc[0].count() #features
    n = train_data[0].count() #samples
    X = train_data.to_numpy() #training data: dim = n x f
    r = np.zeros((n,k)) #membership weights: dim = n x k
    pi = np.zeros((k,1)) #mixture weights: dim = k x 1
    mu = np.zeros((k,f)) #means: dim = k x f
    sigma = np.zeros((f,f,k)) #covars: dim = f x f x k
    nIter = 10 #run EM for 10 iterations
    #initialize sigma, pi, mu
    for c in range(0,k):
        pi[c,:] = 1/k
        mu[c,:] = np.mean(X,axis=0)
        sigma[:,:,c] = np.cov(X,rowvar=False)

In [10]:
print('initial pi: \n', pi,'\n')
print('initial mu: \n', mu,'\n')
print('initial sigma: \n')
for c in range(0,k):
    print(sigma[:,:,c])

initial pi: 
 [[0.33333333]
 [0.33333333]
 [0.33333333]] 

initial mu: 
 [[-0.82042865 -0.58219397]
 [-0.82042865 -0.58219397]
 [-0.82042865 -0.58219397]] 

initial sigma: 

[[ 7.9328436  -0.10559592]
 [-0.10559592  6.2470177 ]]
[[ 7.9328436  -0.10559592]
 [-0.10559592  6.2470177 ]]
[[ 7.9328436  -0.10559592]
 [-0.10559592  6.2470177 ]]


In [11]:
pi,mu,sigma = EM(X,n,k,r,pi,mu,sigma,nIter)

log-likelihood for iteration  0  is:  [-885.43171119]
log-likelihood for iteration  1  is:  [-1521.53552285]
log-likelihood for iteration  2  is:  [-1061.65237995]
log-likelihood for iteration  3  is:  [-890.3059187]
log-likelihood for iteration  4  is:  [-887.41565901]
log-likelihood for iteration  5  is:  [-887.43168481]
log-likelihood for iteration  6  is:  [-887.43218431]
log-likelihood for iteration  7  is:  [-887.43221109]
log-likelihood for iteration  8  is:  [-887.43221153]
log-likelihood for iteration  9  is:  [-887.43221153]


In [12]:
print('final pi is as follows: \n', pi,'\n')
print('final mu is as follows: \n', mu,'\n')
print('final sigma is as follows: \n')
for c in range(0,k):
    print(sigma[:,:,c])

final pi is as follows: 
 [[1.00000000e+00]
 [7.95890073e-17]
 [6.10258298e-17]] 

final mu is as follows: 
 [[-0.82042865 -0.58219397]
 [-0.24552594  1.64909829]
 [-0.24558495  1.64925094]] 

final sigma is as follows: 

[[ 7.92491076 -0.10549033]
 [-0.10549033  6.24077068]]
[[0.19124412 0.09158298]
 [0.09158298 0.12875877]]
[[0.19064662 0.09136423]
 [0.09136423 0.1283201 ]]
